# Auto codificadores Variacionales (VAE)

# Introducción

## Modelo GAN

El objetivo de una red generativa adversaria es escencialmente aproximar la distribución de un modelo de clasificación a partir de la entrada.

En el caso de la base de datos *mnist* el modelo GAN es una función 

$$
g:[-1,1]^{d} \to [0,1]^{k},
$$

en donde, si $\mathbf{x}\in [-1,1]^{d}$ $g(\mathbf{x})$ es una imagen que el discriminador no puede clasificarcomo falsa.


Un modelo GAN digamos $g$ es un  modelo generativo, debido a que a partir de un hipercubo, sobre el cual asumimos una distribuición uniforme,  es capaz de producir una imagen  que es reconocida  por un clasificador de manera correcta, como si fuera una imagen auténtica. 

Esto significa que si $X\sim [-1,1]^{d}$, entonces $g(X) \in \mathcal{C}(c)$, en donde $\mathcal{C}(c)$ es un sunconjunto de 
$[0,1]^{k}$ asociado a la categoría $c$.



## Modelo autoenconder

Por otro lado, en un autoencoder, el encoder hace un sumergimiento (embedding) de  la entrada en $\mathcal{R}^m$.

En el caso de *mnist* el encoder (codficador) es una función 

$$
f : [0,1]^{k} \to \mathcal{R}^m,
$$

en donde en principio $m<<k$.

El decoder (decodificador) es ahora un modelo generativo que intenta recuperar a partir del código $f(\mathcal{x})$ el objeto $\mathcal{x}$. En realidad, el decoder hace un trabajo similar a un modelo GAN, pero desde la represetación latente intermedia. 

# Inferencia Variacional

El propósito de la inferencia variacional es aproximar una densidad haciendo una paso intermedio por un espacio de variables latentes.

El proceso  puede ser imaginado de una forma análoga a la construcción de un codificador en variables latentes.

## Sobre un problema de Medición de trazos latentes.

Para fijar ideas, supongamos que se tiene el resultado de una prueba académica aplicada a una muestra de personas. Más aún, supongamos que la codificación es binaria, en donde 1 (uno) codifica una respuesta correcta y 0 (cero) una respuesta incorrecta. 

Observe que la representación vectorial de la base de datos *mnist*, puede considerarse similar si cada pixel se codifica como cero o uno únicamente.

Entonces a la entrada se tienen vectores binarios  $\mathbf{x}$. de tamaño fijo, digamos $d=100$. Se busca entonces una respresentación de estos vectores en un espacio Euclideano de dimensión $m$, en donde $m<<d$. 

Los expertos utilizan distintas técnicas como por ejemplo el empleo del análisis de componentes principales (ACP) o la $q$-dimensión, para detectar la dimensión $m$.

El objetivo central de la medición es justamente la obtención de los vectores latentes que denotaremos $\mathbf{z}$.




## Planteamiento del problema variacional

Supongamos que $p_{\theta}(\mathbf{x})$ es la densidad asociada a un vector aleatorio de respuesta. El problema estadístico en principio es estimar el parámetro $\theta$ que indexa a la distribución.

Si se asume que $\mathbf{z}$ es el vector latente asociado a $\mathbf{x}$  se tiene que

$$
P_{\theta}(\mathbf{x}) = \int p_{\theta}(\mathbf{x},\mathbf{z})d\mathbf{z}.
$$ 

A partir de esta ecuación se puede escribir que 


$$
P_{\theta}(\mathbf{x})  = \int P_{\theta}(\mathbf{x}|\mathbf{z})P(\mathbf{z})d\mathbf{z},
$$

en donde $P(\mathbf{z})$ es la distribución marginal del vector latente $\mathbf{z}$. Esta expresión muestra el modelo generativo en el problema. Observese que una muestra de la distribución $P_{\theta}(\mathbf{x})$ puede ser obtenido como sigue:

1. Genere una muestra $\mathbf{z}\sim P(\mathbf{z})$
2. Genere una muestra de $P_{\theta}(\mathbf{x}|\mathbf{z})$.

El problema es que en general $P_{\theta}(\mathbf{x}|\mathbf{z})$ es intratable, en el sentido que por un lado la integral no puede obtenerse de forma directa y muestras de $P_{\theta}(\mathbf{x}|\mathbf{z})$  tampoco se obtienen directamente, dado que precisamente se desconoce el parámetro $\theta$. Obviamente al comienzo la marginal $P(\mathbf{z})$ es desconocida

Observe que adicionalmente 

$$
p_{\theta}(\mathbf{x})= P(\mathbf{x})\int P_{\theta}(\mathbf{z}|\mathbf{x})d\mathbf{z}.
$$

 El problema para determinar $P_{\theta}(\mathbf{x})$ es que en general $P_{\theta}(\mathbf{z}|\mathbf{x})$ también es intratable.
 
 

## Aproximación Variacional

Con el propósito de convertir $p_{\theta}(\mathbf{z}|\mathbf{x})$ en una función de densidad tratable la solución propuesta desde la inferencia variacional es la introducción de una densidad aproximada $Q_{\phi}(\mathbf{z}|\mathbf{x})$ de tal manera que

$$
Q_{\phi}(\mathbf{z}|\mathbf{x}) \approx P_{\theta}(\mathbf{z}|\mathbf{x}).
$$

La densidad $Q_{\phi}(\mathbf{z}|\mathbf{x})$ se escoge en una familia de distribuciones tratables indexadas por $\phi$. Es común escoger $Q_{\phi}(\mathbf{z}|\mathbf{x})$ en la familia normal multivariada. Eso haremos en esta lección. 

Entonces para cada $\mathbf{x}$ tendremos que

$$
Q_{\phi}(\mathbf{z}|\mathbf{x}) = \mathcal{N}(\mathbf{z}; \boldsymbol{\mu}(\mathbf{x}), \text{diag}(\boldsymbol{\sigma}(\mathbf{x})^2)
$$

$\boldsymbol{\mu}(\mathbf{x})$ es elvector de medias (condicionadas a la entrada) y $(\boldsymbol{\sigma}(\mathbf{x})$ es un vector de desviaciones estándar. Como la matriz de covarianza es diagonal, se está asumiendo que las componentes del vector $\mathbf{z}$ son condicionalmente independientes, dado el vector de entrada $\mathbf{x}$.

## Divergencia Kullback-Leibler (KL)

Una vez se ha definido la familia de disgtribuciones a partir de la cual se obtendrá la aproximación $Q_{\phi}(\mathbf{z}|\mathbf{x})$ el siguiente paso es deicidir como medir la proximidad o la discrepancia de la densidad aproximante con la densidad original. La solución sugerida desde la inferencia variacional es usar la divergencia KL, la cual se define por

$$
D_{KL}(Q_{\phi}\left(\mathbf{z}|\mathbf{x})|| p_{\theta}(\mathbf{z}|\mathbf{x})\right)  = \mathbb{E}_{\phi}(\log Q_{\phi}\left(\mathbf{z}|\mathbf{x}) - \log p_{\theta}(\mathbf{z}|\mathbf{x})\right)).
$$

El símbolo $\mathbb{E}_{\phi}$ indica que la esperanza es con respecto a la densidad $Q_{\phi}\left(\mathbf{z}|\mathbf{x}\right)$.

# Cota inferior de la evidencia (ELBO)

El objetivo en la inferencia variacional es encontrar una densidad aproximante $Q_{\phi}(\mathbf{z}|\mathbf{x})$  para la densidad $p_{\theta}(\mathbf{z}|\mathbf{x})$ utilizando como métrica la divergencia KL, que por cierto no es una distancia, dado que no es simétrica.

A partir del teorema de Bayes se obtiene que 

$$
P_{\theta}(\mathbf{z}|\mathbf{x})= \frac{P_{\theta}(\mathbf{x}|\mathbf{z})P_{\theta}(\mathbf{z})}{P_{\theta}(\mathbf{x})}
$$

Por lo que la divergencia KL se transforma en

$$
D_{KL}(Q_{\phi}(\mathbf{z}|\mathbf{x})|| p_{\theta}(\mathbf{z}|\mathbf{x}))  = \mathbb{E}_{\phi}(\log Q_{\phi}\left(\mathbf{z}|\mathbf{x}) - \log p_{\theta}(\mathbf{x}|\mathbf{z})- \log P_{\theta}(\mathbf{z})\right)) + \log P_{\theta}(\mathbf{x}).
$$

De donde se obtiene que


$$
\log P_{\theta}(\mathbf{x}) - 
D_{KL}[Q_{\phi}(\mathbf{z}|\mathbf{x})|| p_{\theta}(\mathbf{z}|\mathbf{x})] = \mathbb{E}_{\phi}[\log p_{\theta}(\mathbf{x}|\mathbf{z})]-
D_{KL}[Q_{\phi}(\mathbf{z}|\mathbf{x})|| p_{\theta}(\mathbf{z})]
$$

Esta ecuación constituye lw núcleo de la inferencia variacional. El lado izquierdo de la ecuación  contiene el término $P_{\theta}(\mathbf{x})$ que se busca maximizar menos el error de la aproximación medido por $D_{KL}[Q_{\phi}(\mathbf{z}|\mathbf{x})|| p_{\theta}(\mathbf{z}|\mathbf{x})]$ que se espera que sea aproximadamente cero. 


Se sabe que la divergencia KL siempre es positiva, por lo que la parte izquierda de la ecuación se denomina como la cota inferior de la evidencia (**ELBO**) del inglés *evidence lower bound*.

# Optimización

La ecuación clave d ela inferencia variacional es dada por

$$
\begin{align}
\text{ELBO}  & = 
\log P_{\theta}(\mathbf{x}) - 
D_{KL}[Q_{\phi}(\mathbf{z}|\mathbf{x})|| p_{\theta}(\mathbf{z}|\mathbf{x})] \\
&= \mathbb{E}_{\phi}[\log p_{\theta}(\mathbf{x}|\mathbf{z})]-
D_{KL}[Q_{\phi}(\mathbf{z}|\mathbf{x})|| p_{\theta}(\mathbf{z})]
\end{align}
$$


El proceso de optimización se basa en la segunda parte ecuación. El término $\mathbb{E}_{\phi}[\log p_{\theta}(\mathbf{x}|\mathbf{z})]$ corresponde al modelo generativo en el problema. La interpretación estadística de esta término es que el modelo generador toma muestras obtenidas de la salida del modelo de inferencia $P_{\theta}(\mathbf{z}|\mathbf{x})$, el cual estamos aproximando con $Q_{\phi}(\mathbf{z}|\mathbf{x})$. Es decir, se genera una muestra $\mathbf{z} \sim Q_{\phi}(\mathbf{z}|\mathbf{x})$ y partir de esta se trata de reconstruir la entrada $\mathbf{x}$.

En el ejemplo propuesto, si se considera que se tiene vectores dicotómicos, se asume una distribución de Bernoulli para cada componente. Si las respuestas son condicionalmente independientes dado el vector $\mathbf{z}$ entonces la función de pérdida es la entropía cruzada binaria $\mathcal{L}_R$ dada por

$$
\mathcal{L}_R = \frac{1}{d}\sum_{j=1}^d x_j \log p(\mathbf{w}_j'\mathbf{z} + b_j) + (1-x_j)\log(1-p(\mathbf{w}_j'\mathbf{z} + b_j))
$$


El segundo término $D_{KL}[Q_{\phi}(\mathbf{z}|\mathbf{x})|| p_{\theta}(\mathbf{z})]$ puede ser evaluado directamente. Como asumimos que $Q_{\phi}$ es una distribución Gaussiana y si se tiene en cuenta que típicamente $P_{\theta}(\mathbf{z})= P(\mathbf{z})=\mathcal{N}(\mathbf{0},\mathbf{I})$, se obtiene que 

$$
D_{KL}[Q_{\phi}(\mathbf{z}|\mathbf{x})|| p_{\theta}(\mathbf{z})]= \frac{1}{2} \sum_{j=1}^{d} (1+\log(\sigma_j)^2 - (\mu_j)^2-(\sigma_j)^2)
$$

Tanto $\mu_j$ como $\sigma_j$ son funciones de la entrada $\mathbf{x}$ ue se estiman en el modelo de inferencia.

Para minimizar $\mathcal{L}_{KL} =D_{KL}$, se requiere que $\mu_j\to 0$ y $\sigma_j\to 1$.

En resumen para el problema de inferencia variacional la función de pérdida es dada por

$$
\mathcal{L}_{VAE} = \mathcal{L}_{R} + \mathcal{L}_{KL}
$$

# Autoencoder Variacional

El objetivo del codificador en un autoencoder variacional es aproximar $Q_{\phi}(\mathbf{z}|\mathbf{x})$ mediante una red neuronal profunda.

Tanto la media $\boldsymbol{\mu}(\mathbf{x})$ como el vector de desviaciones  estándar $\boldsymbol{\sigma}(\mathbf{x})$ son estimados por la red neuronal codificadora (encoder).

El decodificador toma muestras latentes $\mathbf{z}$ con el propósito de reconstruir la entrada como $\tilde{\mathbf{x}}$.

# El truco de la reparametrización


Los gradientes de propagación hacia atrás no pueden pasar por el bloque de muestreo estocástico. Si bien está bien tener entradas estocásticas para redes neuronales, no es posible los gradientes para pasar por una capa estocástica. La solución a este problema es eliminar el proceso de muestreo como entrada, como se muestra en el lado derecho de la siguiente figura. 


<figure>
<center>
<img src="./Imagenes/reparametrizacion_truco.png" width="500" height="400" align="center"/>
</center>
<figcaption>
<p style="text-align:center">Reparametrización del Autoencoder Variacional </p>
</figcaption>
</figure>

La muestra  se calcula como:

$$
\text{Muestra} = \boldsymbol{\mu} + \boldsymbol{\epsilon} \odot  \boldsymbol{\sigma}
$$

In [1]:
# install the nb extensions
# conda install -c conda-forge jupyter_contrib_nbextensions 
#
# enable the extensions
#jupyter contrib nbextension install --user
#jupyter nbextension enable equation-numbering/main
#

In [3]:
%%javascript
MathJax.Hub.Config({
    TeX: { equationNumbers: { autoNumber: "AMS" } }
});

MathJax.Hub.Queue(
  ["resetEquationNumbers", MathJax.InputJax.TeX],
  ["PreProcess", MathJax.Hub],
  ["Reprocess", MathJax.Hub]
);

<IPython.core.display.Javascript object>

# Autoencoder Variacional

In [ ]:
'''Example of VAE on MNIST dataset using MLP

The VAE has a modular design. The encoder, decoder and VAE
are 3 models that share weights. After training the VAE model,
the encoder can be used to  generate latent vectors.
The decoder can be used to generate MNIST digits by sampling the
latent vector from a Gaussian distribution with mean=0 and std=1.

# Reference

[1] Kingma, Diederik P., and Max Welling.
"Auto-encoding variational bayes."
https://arxiv.org/abs/1312.6114
'''

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensorflow.keras.layers import Lambda, Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.losses import mse, binary_crossentropy
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K

import numpy as np
import matplotlib.pyplot as plt
import argparse
import os

In [ ]:
# reparameterization trick
# instead of sampling from Q(z|X), sample eps = N(0,I)
# z = z_mean + sqrt(var)*eps
def sampling(args):
    """Reparameterization trick by sampling 
        fr an isotropic unit Gaussian.

    # Arguments:
        args (tensor): mean and log of variance of Q(z|X)

    # Returns:
        z (tensor): sampled latent vector
    """

    z_mean, z_log_var = args
    # K is the keras backend
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

In [ ]:
def plot_results(models,
                 data,
                 batch_size=128,
                 model_name="vae_mnist"):
    """Plots labels and MNIST digits as function 
        of 2-dim latent vector

    # Arguments:
        models (tuple): encoder and decoder models
        data (tuple): test data and label
        batch_size (int): prediction batch size
        model_name (string): which model is using this function
    """

    encoder, decoder = models
    x_test, y_test = data
    xmin = ymin = -4
    xmax = ymax = +4
    os.makedirs(model_name, exist_ok=True)

    filename = os.path.join(model_name, "vae_mean.png")
    # display a 2D plot of the digit classes in the latent space
    z, _, _ = encoder.predict(x_test,
                              batch_size=batch_size)
    plt.figure(figsize=(12, 10))

    # axes x and y ranges
    axes = plt.gca()
    axes.set_xlim([xmin,xmax])
    axes.set_ylim([ymin,ymax])

    # subsample to reduce density of points on the plot
    z = z[0::2]
    y_test = y_test[0::2]
    plt.scatter(z[:, 0], z[:, 1], marker="")
    for i, digit in enumerate(y_test):
        axes.annotate(digit, (z[i, 0], z[i, 1]))
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.savefig(filename)
    plt.show()

    filename = os.path.join(model_name, "digits_over_latent.png")
    # display a 30x30 2D manifold of digits
    n = 30
    digit_size = 28
    figure = np.zeros((digit_size * n, digit_size * n))
    # linearly spaced coordinates corresponding to the 2D plot
    # of digit classes in the latent space
    grid_x = np.linspace(-4, 4, n)
    grid_y = np.linspace(-4, 4, n)[::-1]

    for i, yi in enumerate(grid_y):
        for j, xi in enumerate(grid_x):
            z_sample = np.array([[xi, yi]])
            x_decoded = decoder.predict(z_sample)
            digit = x_decoded[0].reshape(digit_size, digit_size)
            figure[i * digit_size: (i + 1) * digit_size,
                   j * digit_size: (j + 1) * digit_size] = digit

    plt.figure(figsize=(10, 10))
    start_range = digit_size // 2
    end_range = n * digit_size + start_range + 1
    pixel_range = np.arange(start_range, end_range, digit_size)
    sample_range_x = np.round(grid_x, 1)
    sample_range_y = np.round(grid_y, 1)
    plt.xticks(pixel_range, sample_range_x)
    plt.yticks(pixel_range, sample_range_y)
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.imshow(figure, cmap='Greys_r')
    plt.savefig(filename)
    plt.show()

In [ ]:
# MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

image_size = x_train.shape[1]
original_dim = image_size * image_size
x_train = np.reshape(x_train, [-1, original_dim])
x_test = np.reshape(x_test, [-1, original_dim])
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

In [ ]:
# network parameters
input_shape = (original_dim, )
intermediate_dim = 512
batch_size = 128
latent_dim = 2
epochs = 50

In [ ]:
# VAE model = encoder + decoder
# build encoder model
inputs = Input(shape=input_shape, name='encoder_input')
x = Dense(intermediate_dim, activation='relu')(inputs)
z_mean = Dense(latent_dim, name='z_mean')(x)
z_log_var = Dense(latent_dim, name='z_log_var')(x)

In [ ]:
# use reparameterization trick to push the sampling out as input
# note that "output_shape" isn't necessary 
# with the TensorFlow backend
z = Lambda(sampling,
           output_shape=(latent_dim,), 
           name='z')([z_mean, z_log_var])

In [ ]:
# instantiate encoder model
encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
encoder.summary()
plot_model(encoder,
           to_file='vae_mlp_encoder.png',
           show_shapes=True)

In [ ]:
# build decoder model
latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
x = Dense(intermediate_dim, activation='relu')(latent_inputs)
outputs = Dense(original_dim, activation='sigmoid')(x)

# instantiate decoder model
decoder = Model(latent_inputs, outputs, name='decoder')
decoder.summary()
plot_model(decoder,
           to_file='vae_mlp_decoder.png', 
           show_shapes=True)

In [ ]:
# instantiate VAE model
outputs = decoder(encoder(inputs)[2])
vae = Model(inputs, outputs, name='vae_mlp')

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    help_ = "Load tf model trained weights"
    parser.add_argument("-w", "--weights", help=help_)
    help_ = "Use binary cross entropy instead of mse (default)"
    parser.add_argument("--bce", help=help_, action='store_true')
    args = parser.parse_args()
    models = (encoder, decoder)
    data = (x_test, y_test)

    # VAE loss = mse_loss or xent_loss + kl_loss
    if args.bce:
        reconstruction_loss = binary_crossentropy(inputs,
                                                  outputs)
    else:
        reconstruction_loss = mse(inputs, outputs)

    reconstruction_loss *= original_dim
    kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
    kl_loss = K.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    vae_loss = K.mean(reconstruction_loss + kl_loss)
    vae.add_loss(vae_loss)
    vae.compile(optimizer='adam')
    vae.summary()
    plot_model(vae,
               to_file='vae_mlp.png',
               show_shapes=True)

    save_dir = "vae_mlp_weights"
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    if args.weights:
        filepath = os.path.join(save_dir, args.weights)
        vae = vae.load_weights(filepath)
    else:
        # train the autoencoder
        vae.fit(x_train,
                epochs=epochs,
                batch_size=batch_size,
                validation_data=(x_test, None))
        filepath = os.path.join(save_dir, 'vae_mlp_mnist.tf')
        vae.save_weights(filepath)

    plot_results(models,
                 data,
                 batch_size=batch_size,
                 model_name="vae_mlp")